In [1]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    from io import StringIO
    import time

    def get_day_price(code, max_pages=353):
        base_url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
        headers = {"User-Agent": "Mozilla/5.0"}
        df_total = pd.DataFrame()

        for page in range(1, max_pages + 1):
            url = f"{base_url}&page={page}"
            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"페이지 {page} 요청 실패: 상태 코드 {res.status_code}")
                break

            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("table", class_="type2")
            if table is None:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                break

            df = pd.read_html(StringIO(str(table)), header=0)[0]
            df = df.dropna()

            # 👉 만약 페이지에 데이터가 없는 경우 자동 종료
            if df.empty:
                print(f"\n✅ 데이터 없음: 페이지 {page}에서 종료합니다.")
                break

            # 전일비 처리
            def parse_change(val):
                val = str(val).strip().replace(",", "")
                if val.startswith("상승"):
                    return int(val.replace("상승", "").strip())
                elif val.startswith("하락"):
                    return -int(val.replace("하락", "").strip())
                elif val.startswith("보합"):
                    return 0
                else:
                    return None

            df["전일비"] = df["전일비"].apply(parse_change)

            # 거래량 처리
            df["거래량"] = df["거래량"].astype(str).str.replace(",", "")
            df["거래량"] = df["거래량"].apply(lambda x: int(float(x)))

            # ✅ 출력 – 첫 페이지만 헤더 포함
            if page == 1:
                print(df.to_string(index=False))  # 헤더 포함
            else:
                print(df.to_string(index=False, header=False))  # 헤더 생략

            df_total = pd.concat([df_total, df], ignore_index=True)
            time.sleep(0.5)

        return df_total

    # 종목 코드 및 기업명 설정
    code = "082740"
    company_name = "한화엔진"

    # 데이터 수집
    df = get_day_price(code, max_pages=353)

    # 컬럼명 정리
    df.columns = ["날짜", "종가", "전일비", "시가", "고가", "저가", "거래량"]

    # 날짜 형식 변환
    df["날짜"] = pd.to_datetime(df["날짜"])

    # ✅ 날짜 기준으로 중복 제거
    df = df.drop_duplicates(subset="날짜", keep="first")

    # 날짜 내림차순 정렬
    df = df.sort_values("날짜", ascending=False)

    # CSV 파일로 저장
    filename = f"{company_name}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ {filename} 파일이 저장되었습니다.")


        날짜      종가  전일비      시가      고가      저가     거래량
2025.04.25 27150.0  650 26900.0 27500.0 26600.0 2121725
2025.04.24 26500.0  150 26450.0 26750.0 25950.0 1161603
2025.04.23 26350.0  500 26150.0 26850.0 25500.0 1475342
2025.04.22 25850.0 1450 24000.0 26100.0 24000.0 2825990
2025.04.21 24400.0 -750 25450.0 25600.0 24200.0  770094
2025.04.18 25150.0 -100 25550.0 25550.0 24650.0  593002
2025.04.17 25250.0  500 24850.0 25400.0 24500.0  689639
2025.04.16 24750.0 -750 24950.0 25425.0 24550.0  923534
2025.04.15 25500.0  300 25650.0 25650.0 24650.0 1209545
2025.04.14 25200.0 1300 24900.0 25400.0 24500.0 1589397
2025.04.11 23900.0  1100 23100.0 24100.0 22950.0 1685618
2025.04.10 22800.0  1400 22900.0 22950.0 22250.0  584422
2025.04.09 21400.0  -400 22600.0 22650.0 21150.0  895034
2025.04.08 21800.0  1800 20950.0 21900.0 20800.0  971845
2025.04.07 20000.0 -2250 21300.0 21450.0 20000.0  923755
2025.04.04 22250.0  -250 22250.0 22900.0 21500.0  906589
2025.04.03 22500.0     0 21900.0 22850.0 2

C:\Users\human\AppData\Local\Temp\ipykernel_5316\965010956.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df], ignore_index=True)



✅ 한화엔진.csv 파일이 저장되었습니다.
